In [2]:
import pandas as pd
from pathlib import Path

In [4]:
folder = Path(r"C:\Users\add41\Documents\Data_Engineer\Project\example_data\20251119_MySQL_backup\TPEI")

file = "restaurant_TPEI_place_id.csv"

path = folder / file

df = pd.read_csv(path)

df.head(15)

,name,place_id,buss_status,update_time,in_boundary,update_date
0,狂魔炒飯x餓鬼糧倉-文山店,ChIJzw0SIDQBaDQRoVAinuv2Sqw,OPERATIONAL,2025/11/16 17:23:29,True,2025/11/16
1,擺渡冰室,ChIJZ-xSZQ-rQjQRSYbXgt0Ka9A,OPERATIONAL,2025/11/16 17:23:29,True,2025/11/16
2,貓蕊－貓咪餐廳,ChIJcfSStZOrQjQR3Z8SsPMBkfY,OPERATIONAL,2025/11/16 17:23:29,True,2025/11/16
3,利庵咖哩 景美店 - 台北人氣漢堡排咖哩店 - 香料歐風咖哩 寵物友善 現金only,ChIJx7Ij2EirQjQRviRSPvqWplA,OPERATIONAL,2025/11/16 17:23:29,True,2025/11/16
4,徙巷小餐酒,ChIJiRDpRsurQjQRcHk6v2b4YPM,OPERATIONAL,2025/11/16 17:23:29,True,2025/11/16
5,飛起咖咖Fly High CURRY×CAFÉ 咖哩×咖啡(公休日請見粉專.IG｜寵物友善),ChIJ2f2uZkSrQjQRQkyLoMW8d44,OPERATIONAL,2025/11/16 17:23:29,True,2025/11/16
6,Purrson Bistro,ChIJURTE5n2rQjQRgj-n9IMohFU,OPERATIONAL,2025/11/16 17:23:29,True,2025/11/16
7,Juicy Bun Burger 就是棒 美式餐廳 政大店,ChIJ3ZnMlHuqQjQRMj4lIpQR6xE,OPERATIONAL,2025/11/16 17:23:29,True,2025/11/16
8,LAX,ChIJXRobUI4BaDQRTcT-x-lrF48,OPERATIONAL,2025/11/16 17:23:29,True,2025/11/16
9,自由·之丘,ChIJ_6pKpbSrQjQR1jF-we5DZtk,OPERATIONAL,2025/11/16 17:23:29,True,2025/11/16


In [5]:
folder = Path(r"C:\Users\add41\Documents\Data_Engineer\Project\example_data\20251119_MySQL_backup\TPEI")

file = "20251116supplies_TPEI_finish.csv"

path = folder / file

df = pd.read_csv(path)

df.head(15)

,id,name,buss_status,loc_id,address,phone,op_hours,category_id,rating,rating_total,newest_review,longitude,latitude,map_url,website,place_id,update_time
0,supl1876,咕咕G寵物城-木柵店,OPERATIONAL,TPE008,116臺灣臺北市文山區興隆路四段70-1號,229398833.0,70.0,5,4.1,261.0,2025-04-19,121.561562,24.984316,https://maps.google.com/?cid=9590099804699171261,NaN,ChIJVVVVVd4BaDQRvRmjO_bgFoU,2025/11/16 17:31:46
1,supl1877,PetPark,OPERATIONAL,TPE008,11674臺灣臺北市文山區羅斯福路六段302號,229317200.0,63.0,5,4.6,232.0,2021-05-05,121.539704,24.990713,https://maps.google.com/?cid=707242357215644003,http://www.wonderpet.asia/,ChIJqaK_YQCqQjQRY8nyPy2h0Ak,2025/11/16 17:31:46
2,supl1878,木星寵物,OPERATIONAL,TPE008,116臺灣臺北市文山區木新路三段226號,229368099.0,70.0,5,4.8,64.0,2025-04-28,121.559249,24.981490,https://maps.google.com/?cid=78979114800680598,NaN,ChIJaRgNdAwBaDQRlo5eIhaXGAE,2025/11/16 17:31:46
3,supl1879,PetPark,OPERATIONAL,TPE008,116臺灣臺北市文山區木新路三段272號,229372900.0,63.0,5,4.4,189.0,2025-03-23,121.558347,24.981230,https://maps.google.com/?cid=469218073759243828,https://www.wonderpet.asia/,ChIJ6apPwOABaDQRNN6cIFr_ggY,2025/11/16 17:31:46
4,supl1880,晶晶寵物店（原金美滿寵物）,OPERATIONAL,TPE008,116臺灣臺北市文山區興隆路三段261號,222393188.0,72.0,5,4.3,55.0,2021-04-28,121.558864,24.992368,https://maps.google.com/?cid=13521194320155065958,NaN,ChIJV_b0xAyqQjQRZj7fh5jupLs,2025/11/16 17:31:46
5,supl1881,哈哈窩寵物精品館-北市景興店,OPERATIONAL,TPE008,116臺灣臺北市文山區景興路169號,229337700.0,59.5,5,4.2,68.0,2021-07-06,121.544583,24.993163,https://maps.google.com/?cid=10431518333823885977,https://www.facebook.com/hahahouse107/,ChIJSRm-kAaqQjQRmSKM86cyxJA,2025/11/16 17:31:46
6,supl1882,喜羊羊寵物生活館(北市景興店),OPERATIONAL,TPE008,116臺灣臺北市文山區景興路169號,229337700.0,77.0,5,5.0,2.0,2024-09-05,121.544539,24.993116,https://maps.google.com/?cid=9567540067286104302,http://www.cyypet.com.tw/,ChIJj0CozxWrQjQR7rjnDAC7xoQ,2025/11/16 17:31:46
7,supl1883,豆恩凱寵物沙龍,OPERATIONAL,TPE008,116臺灣臺北市文山區木新路二段1號,286612369.0,54.0,5,4.5,104.0,2025-02-12,121.570835,24.987609,https://maps.google.com/?cid=12388809864078626836,NaN,ChIJ4SSyQHWqQjQRFBBDwu7k7as,2025/11/16 17:31:46
8,supl1884,黛希寵物美容沙龍,OPERATIONAL,TPE008,116臺灣臺北市文山區景華街18號1樓,289319906.0,60.0,5,4.8,77.0,2024-08-30,121.542148,24.994844,https://maps.google.com/?cid=9574317886448112929,https://line.me/R/ti/p/@zfz5058g,ChIJfe1t-wOqQjQRIZVoDGTP3oQ,2025/11/16 17:31:46
9,supl1885,景興動物醫院,OPERATIONAL,TPE008,116臺灣臺北市文山區景華街92號,229330600.0,60.0,5,4.5,222.0,2025-08-24,121.545716,24.995151,https://maps.google.com/?cid=12713671958385079329,NaN,ChIJ6Td9tQWqQjQRITRjRkkJcLA,2025/11/16 17:31:46
